In [1]:
#テキストファイルを読み込んで、形態素解析
#ファイルはcsv(UTF-8)、識別フラグ・テキスト・除外単語の3項目
#除外項目は必要に応じ個別指定
#識別フラグ×単語数、識別フラグ×共起パターンをファイル出力
#指定した識別タグについて、ワードクラウド、共起ネットワークを出力・表示
#各識別タグごとに、ワードクラウド、共起ネットワークを出力・表示

In [2]:
#ライブラリのインポート
import pandas as pd
import numpy as np
import re
import csv
import MeCab
import fitz
import sys
sys.path.append('/work_dir') 
import talknize_module_20240909 as tk


In [3]:
#分析対象ファイル読込み

directory_name="data/"
file_name= 'sample.csv'
target = directory_name + file_name

#CSVファイルのヘッダ有無を指定 ヘッダあり：1,ヘッダなし0
header_flag=1

#ヘッダ有無をふまえてファイル読込み
if header_flag==1:
    #ヘッダ情報あり
    df_targets = pd.read_csv(target, dtype=str)
else:
    #ヘッダ情報なし
    df_targets = pd.read_csv(target, header=None, dtype=str)
    df_targets.columns = ['flag', 'text', 'exclusion']# データフレームのカラム名を後付けで定義

df_targets.head()
#print(df_targets.dtypes)

,flag,text,exclusion
0,Text_a,貴社が汽車で帰社した。蛙の子は蛙。,adm
1,Text_b,隣の客はよく柿食う客だ。柿といえば秋の果物\n柿食えば金がなるなり法隆寺。,NaN


In [4]:
#形態素解析の前処理
#分析対象データの特性に対応するため、前処理はモジュールではなくコードとして記載
#talknize_module.pyにも標準的な処理も記載し、適宜使い分け可能にする

#----------------------------------------------------------------------
#テキストファイルの各行に記載された文字列を、処理用文字列として整形・リスト化
def file_to_list(file_path):

    # 空のリストを作成
    return_list = []
    try:
        # 指定されたファイルを読み込み、各行をリストに追加
        with open(file_path, 'r') as file:
            # ファイル内の各行をループし、行末の改行や余分な空白を除去してリストに格納
            return_list = [line.strip() for line in file]
    # ファイルが存在しない場合は例外を無視する
    except FileNotFoundError:
        pass
    # リストを返す
    return return_list

#----------------------------------------------------------------------
#形態素解析前のテキストデータ処理（
#形態素解析の前に、無駄な記号やヘッダ・フッタ等の文言をテキストから除外
def pre_tk(text):

    replaced_text = text

    #exclusion_list処理前に処理する必要のあるもの
    #【特例処理】除外処理前に、文頭のこれら記号は「箇条書き」とみなし、続く文言を一文として扱う
    replaced_text = re.sub(r'[■□▪▫▲△▶▷▸▹▼▽◆◇●〇]', '。\n', replaced_text)
    #replaced_text = re.sub(r'[〇●◇◆□■▶△▲▽▼▫▪▹▶▸]', '', replaced_text)#上記以外は除去

    exclusion_list = []    
    exclusion_file1 = "userdic/exclusion_phrases1.txt"  # 各企業の除外フレーズを記載したリスト
    #exclusion_file2 = "userdic/exclusion_phrases2.txt"  # 各企業の除外フレーズを記載したリスト2（pageinfoから都度取り込み）
    #exclusion_file3 = "userdic/exclusion_codes.txt"  # その他記号・年月日・URL等を除外するためのリスト
    exclusion_list = file_to_list(exclusion_file1)# + fileto_list(exclusion_file2)+ file_to_list(exclusion_file3)

    for pattern in exclusion_list:
        replaced_text = re.sub(pattern, ' ', replaced_text)

    return replaced_text


In [5]:
#形態素解析の後処理
#形態素解析結果（tokenリスト）から、ストップワード、特定の条件の文字列等を除外
#import fitz
#import re
from collections import OrderedDict
import re

def post_tk(tokens):
    
    replaced_list = tokens    

    # stopwords（ファイルに格納）を除去
    path_stopwords = "userdic/stopwords.txt"
    stopwords = file_to_list(path_stopwords)
    stopwords = list(OrderedDict.fromkeys(stopwords)) # 元の順序を保持しつつ、重複を除去（# Python 3.7以降）
    replaced_list = [t for t in replaced_list if t not in stopwords]
    
    # ひらがなのみの要素を除去
    kana_re = re.compile("^[ぁ-ゖ]+$")
    replaced_list = [t for t in replaced_list if not kana_re.match(t)]

    # アルファベット1文字のみの要素を除去
    alphabet_re = re.compile("^[a-zA-Z]$")
    replaced_list = [t for t in replaced_list if not alphabet_re.match(t)]

    #特定の形態の数値要素を除去
    number_re = re.compile("^[\d,]+")
    replaced_list = [t for t in replaced_list if not number_re.match(t)]

    
    return replaced_list

In [6]:
#リストを受けて除外する処理を関数化
def post_tk_sp(token, exclusion_str):
    replaced_list = token

    # exclusion_strが文字列かどうかを確認し、そうでない場合は空リストにする
    if isinstance(exclusion_str, str):
        exclusion_list = exclusion_str.split("\n")
    else:
        exclusion_list = []

    # exclusion_listにないものだけを出力
    replaced_list = [item for item in replaced_list if item not in exclusion_list]

    return replaced_list


In [7]:
#抽出単語によるワードクラウド作成
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

#単語と件数のデータフレームをインプットとして、ワードクラウドを出力
def WCFreq(flag_string, df_wordcount, plotflag):
    # 日本語フォントのパスを指定
    jp_font_path = '/usr/share/fonts/opentype/ipaexfont-gothic/ipaexg.ttf'
    
    # ワードクラウドのフォーマット指定
    wordcloud = WordCloud(width=800, height=400, background_color='white',font_path=jp_font_path)
    # 単語とその頻度を辞書形式に変換
    word_freq = {word: freq for word, freq in zip(df_wordcount['単語'], df_wordcount['件数'])}
    # ワードクラウドの生成
    wordcloud.generate_from_frequencies(word_freq)
    
    # プロット
    if plotflag==True:
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.show()
    
    # 結果をpngファイルに出力
    file_name_wordcloud=f'output/{flag_string}_WordCloud.png'
    wordcloud.to_file(file_name_wordcloud)
    

In [8]:
#flag単位で、形態素解析と集計結果を出力

# flagとtokenのみのDataFrameも用意しておく
df_tokens = pd.DataFrame(columns=['flag', 'token'])

for idx, row in df_targets.iterrows():
    flag_value = row.flag
    text_value = row.text
    exclusion_value = row.exclusion

    # token_listの作成
    token_list = [flag_value, post_tk_sp(post_tk(tk.mecab_tokenizer(pre_tk(text_value))), exclusion_value)]

    # 新しい行をpd.Seriesとして追加
    new_row = pd.Series(token_list, index=df_tokens.columns)

    # pd.concatを使ってDataFrameに新しい行を追加
    df_tokens = pd.concat([df_tokens, new_row.to_frame().T], ignore_index=True)

print(df_tokens)


     flag                            token
0  Text_a              [貴社, 汽車, 帰社, 蛙の子は蛙]
1  Text_b  [隣, 客, 柿, 客, 柿, 秋の, 果物, 柿, 法隆寺]


In [9]:
#flag単位で、形態素解析と集計結果を出力
#csvを読み込んだDataFrameに、形態素解析結果を列で追加

#読み込んだデータフレームに'token'を追加
df_targets['token']=''

for idx, row in df_targets.iterrows():
    flag_value = row.flag
    text_value = row.text
    exclusion_value = row.exclusion

    # token_listの作成
    token_list = post_tk_sp(post_tk(tk.mecab_tokenizer(pre_tk(text_value))), exclusion_value)

    # 各行に対応するトークンを代入
    df_targets.at[idx, 'token'] = token_list

df_targets
df_targets[['flag','token']]

,flag,token
0,Text_a,"[貴社, 汽車, 帰社, 蛙の子は蛙]"
1,Text_b,"[隣, 客, 柿, 客, 柿, 秋の, 果物, 柿, 法隆寺]"


In [10]:
#DataFrameの行ごとに、形態素の件数を集計
from collections import Counter

word_count = pd.DataFrame(columns=['flag','単語','件数'])

for idx, row in df_targets.iterrows():
    flag_value = row.flag
    text_value = row.text
    exclusion_value = row.exclusion
    token_value = row.token

    # 辞書形式で単語をカウント
    counter = Counter(token_value)
    # 単語、件数をDataFrameに格納
    count_df = pd.DataFrame(list(counter.items()), columns=['単語', '件数'])
    # DataFrameを件数でソート
    count_df = count_df.sort_values(by='件数', ascending=False)

    # flagを加えて集計結果のデータフレームに格納
    count_df['flag'] = flag_value
    columns_order = ['flag', '単語', '件数']
    count_df = count_df[columns_order]
    word_count = pd.concat([word_count, count_df], ignore_index=True)

    # flag単位でファイル出力
    file_name_word_count = f"output/{flag_value}_Word_Count.csv"
    count_df.to_csv(file_name_word_count, encoding="utf_8_sig", index=False)    
    
    #定義したモジュールでWordCloudも出力
    WCFreq(flag_value, count_df, False)

word_count  


,flag,単語,件数
0,Text_a,貴社,1
1,Text_a,汽車,1
2,Text_a,帰社,1
3,Text_a,蛙の子は蛙,1
4,Text_b,柿,3
5,Text_b,客,2
6,Text_b,隣,1
7,Text_b,秋の,1
8,Text_b,果物,1
9,Text_b,法隆寺,1


In [11]:
#共起分析
#テキストを「。」or「\n」で区切り、文章単位のリストとして格納
import itertools

pair_count =  pd.DataFrame(columns=['flag','単語1','単語2','件数'])

for idx, row in df_targets.iterrows():
    flag_value = row.flag
    text_value = row.text
    exclusion_value = row.exclusion
    token_value = row.token

    # テキストデータを「。」or「\n」で区切ったものを sentence とし、それを形態素解析したものを格納
    sentences = re.split(r'[。\n]', pre_tk(text_value))  # 文を分割
    tokenized_sentences = []

    for sentence in sentences:
        tokens = tk.mecab_tokenizer(sentence)  # 形態素解析
        if isinstance(tokens, list):  # リストで返ってくることを確認
            cleaned_tokens = post_tk(tokens)  # post_tkでトークンをフィルタリング
            final_tokens = post_tk_sp(cleaned_tokens, exclusion_value)  # exclusion_valueでフィルタリング
            tokenized_sentences.append(final_tokens)

    #print(tokenized_sentences)

    #各文中の、形態素組み合わせを作る
    token_pairs = [list(itertools.combinations(token,2)) for token in tokenized_sentences]
    #print(token_combs)
    
    #組み合わせた2つの形態素の並びをソート
    sorted_pairs = [[tuple(sorted(token)) for token in pair] for pair in token_pairs]
    #for pair in token_combs: token_combs内の各文に対して処理
    #for pair in token_comb: 各文内の各組み合わせ（例えば、(a, b)）に対して処理を行います。
    #sorted(token): 各組み合わせ（タプル）内のトークンをソート
    #tuple(sorted(token)): ソートされたトークンをタプルに変換（リストがタプルに変換されることで、タプル内の順序を保証）
    #print(sorted_combs)

    target_pairs = []
    for pair in sorted_pairs:
        target_pairs.extend(pair)
    ct = Counter(target_pairs)
    #print(ct)
    
    # 上位から取得する件数を制限する場合は、数値を指定
    most_common_items = ct.most_common()
    pair_df = pd.DataFrame([{"単語1" : i[0][0], "単語2": i[0][1], "件数":i[1]} for i in most_common_items])

    #共起カウントのデータフレームにflagとともに格納
    
    # flagを加えて集計結果のデータフレームに格納
    pair_df['flag'] = flag_value
    columns_order = ['flag', '単語1', '単語2', '件数']
    pair_df = pair_df[columns_order]
    pair_count = pd.concat([pair_count, pair_df], ignore_index=True)
    
    #flag単位でファイル出力
    file_name_pair = f"output/{flag_value}_Co_Occurrence.csv"
    pair_df.to_csv(file_name_pair, encoding="utf_8_sig", index=False)

pair_count


,flag,単語1,単語2,件数
0,Text_a,汽車,貴社,1
1,Text_a,帰社,貴社,1
2,Text_a,帰社,汽車,1
3,Text_b,客,隣,2
4,Text_b,客,柿,2
5,Text_b,柿,隣,1
6,Text_b,客,客,1
7,Text_b,柿,秋の,1
8,Text_b,果物,柿,1
9,Text_b,果物,秋の,1


In [12]:
#ネットワーク分析の下準備
import networkx as nx
from networkx.algorithms.community import girvan_newman
import network_plot_module as npm
import json

#########################################################
# 分析対象とする共起単語の組み合わせ（ノード）の上位件数を指定
analyzed_links=350

# 各flagごとにノードの上位件数までを抽出
pair_count['順位'] = pair_count.groupby('flag')['件数'].rank(method='dense', ascending=False)

limited_df = pair_count[pair_count['順位'] <= analyzed_links]
#########################################################


# flag単位でネットワーク分析

unified_centralities =  pd.DataFrame(columns=[
        'flag',
        'Degree Centrality',
        'Betweenness Centrality',
        'Closeness Centrality',
        'Eigenvector Centrality',
        'Katz Centrality',
        'Community'])


for idx, row in df_targets.iterrows():
    flag_value = row.flag
    text_value = row.text
    exclusion_value = row.exclusion
    token_value = row.token

    network_df = limited_df[limited_df['flag'] == flag_value]
    # DataFrameからネットワークのグラフオブジェクトを作成
    G = nx.from_pandas_edgelist(network_df, '単語1', '単語2', ['件数'])
    # ノードを確認
    #print(G.nodes)
    # エッジを確認
    #print(G.edges(data=True))  # エッジとその属性（件数）も表示

    # 各ノードの中心性を計算
    try:
        degree_centrality = nx.degree_centrality(G)
    except:
        degree_centrality = {node: '' for node in G.nodes()}
        
    try:
        betweenness_centrality = nx.betweenness_centrality(G)
    except:
        betweenness_centrality = {node: '' for node in G.nodes()}
    
    try:
        closeness_centrality = nx.closeness_centrality(G)
    except:
        closeness_centrality = {node: '' for node in G.nodes()}
    
    try:
        eigenvector_centrality = nx.eigenvector_centrality(G)
    except:
        eigenvector_centrality = {node: '' for node in G.nodes()}
    
    try:
        katz_centrality = nx.katz_centrality(G)
    except:
        katz_centrality = {node: '' for node in G.nodes()}
    
    # Girvan-Newmanアルゴリズムでコミュニティに分割
    comp = girvan_newman(G)
    communities = tuple(sorted(c) for c in next(comp))
    
    # 各ノードがどのコミュニティに属するかを記録
    community_map = {}
    for i, community in enumerate(communities):
        for node in community:
            community_map[node] = i
    
    # 中心性を新しいデータフレームに格納
    centrality_df = pd.DataFrame({
        'flag': flag_value,
        'Node': list(G.nodes()),
        'Degree Centrality': [degree_centrality[node] for node in G.nodes()],
        'Betweenness Centrality': [betweenness_centrality[node] for node in G.nodes()],
        'Closeness Centrality': [closeness_centrality[node] for node in G.nodes()],
        'Eigenvector Centrality': [eigenvector_centrality[node] for node in G.nodes()],
        'Katz Centrality': [katz_centrality[node] for node in G.nodes()],
        'Community': [community_map[node] for node in G.nodes()]  # コミュニティ情報を追加
        })
    #print(centrality_df)

    unified_centralities = pd.concat([unified_centralities, centrality_df], ignore_index=True)
    
    file_name_comb = f"output/{flag_value}_Centrality_{analyzed_links}.csv"
    centrality_df.to_csv(file_name_comb, encoding="utf_8_sig", index=False)    

    

In [14]:
#ネットワーク分析の下準備
import networkx as nx
from networkx.algorithms.community import girvan_newman
import network_plot_module as npm
import json

#########################################################
# 分析対象とする共起単語の組み合わせ（ノード）の上位件数を指定
analyzed_links=350

# 各flagごとにノードの上位件数までを抽出
pair_count['順位'] = pair_count.groupby('flag')['件数'].rank(method='dense', ascending=False)

limited_df = pair_count[pair_count['順位'] <= analyzed_links]
#########################################################


# flag単位でネットワーク分析

unified_centralities =  pd.DataFrame(columns=[
        'flag',
        'Degree Centrality',
        'Betweenness Centrality',
        'Closeness Centrality',
        'Eigenvector Centrality',
        'Katz Centrality',
        'Community'])


for idx, row in df_targets.iterrows():
    flag_value = row.flag
    text_value = row.text
    exclusion_value = row.exclusion
    token_value = row.token

    network_df = limited_df[limited_df['flag'] == flag_value]

    # DataFrameからネットワークのグラフオブジェクトを作成
    G = nx.from_pandas_edgelist(network_df, '単語1', '単語2', ['件数'])
    #print(G.nodes) # ノードを確認
    #print(G.edges(data=True))  # エッジとその属性（件数）も表示

    # 各ノードの中心性を計算
    try:
        degree_centrality = nx.degree_centrality(G)
    except:
        degree_centrality = {node: '' for node in G.nodes()}
        
    try:
        betweenness_centrality = nx.betweenness_centrality(G)
    except:
        betweenness_centrality = {node: '' for node in G.nodes()}
    
    try:
        closeness_centrality = nx.closeness_centrality(G)
    except:
        closeness_centrality = {node: '' for node in G.nodes()}
    
    try:
        eigenvector_centrality = nx.eigenvector_centrality(G)
    except:
        eigenvector_centrality = {node: '' for node in G.nodes()}
    
    try:
        katz_centrality = nx.katz_centrality(G)
    except:
        katz_centrality = {node: '' for node in G.nodes()}
    
    # Girvan-Newmanアルゴリズムでコミュニティに分割
    comp = girvan_newman(G)
    communities = tuple(sorted(c) for c in next(comp))
    
    # 各ノードがどのコミュニティに属するかを記録
    community_map = {}
    for i, community in enumerate(communities):
        for node in community:
            community_map[node] = i
    
    # 中心性を新しいデータフレームに格納
    centrality_df = pd.DataFrame({
        'flag': flag_value,
        'Node': list(G.nodes()),
        'Degree Centrality': [degree_centrality[node] for node in G.nodes()],
        'Betweenness Centrality': [betweenness_centrality[node] for node in G.nodes()],
        'Closeness Centrality': [closeness_centrality[node] for node in G.nodes()],
        'Eigenvector Centrality': [eigenvector_centrality[node] for node in G.nodes()],
        'Katz Centrality': [katz_centrality[node] for node in G.nodes()],
        'Community': [community_map[node] for node in G.nodes()]  # コミュニティ情報を追加
        })
    #print(centrality_df)

    unified_centralities = pd.concat([unified_centralities, centrality_df], ignore_index=True)
    
    file_name_comb = f"output/{flag_value}_Centrality_{analyzed_links}.csv"
    centrality_df.to_csv(file_name_comb, encoding="utf_8_sig", index=False)    


    #graph用のDataframeサブセットを作成
    graph_df = network_df[['単語1', '単語2', '件数']]
    #print(graph_df)
    
    #ネットワーク図を描画、ファイル出力
    got_net = npm.kyoki_word_network(graph_df)
    #フィルタボタンを表示させる場合は、set_optionを無効にする必要あり
    #got_net.show_buttons(filter_=['physics'])
    got_net.set_options("""
    const options = {
      "physics": {
        "forceAtlas2Based": {
          "centralGravity": 0.1,
          "springLength": 25,
          "springConstant": 0.1
        },
        "minVelocity": 0.75,
        "solver": "forceAtlas2Based"
      }
    }
    """)
    file_name_kyoki = f'output/{flag_value}_kyoki_{analyzed_links}.html'
    got_net.show(file_name_kyoki)
